In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import mean_squared_error
from tqdm.notebook import tqdm
import umap
from sklearn.manifold import Isomap

In [2]:
from experiments import *

In [9]:
import wandb
wandb.init(project="manifold learning encode-decode")

In [4]:
def umap_experiments():
    for experiment in tqdm(common_experiments, position=0):
        traj = experiment.single_trajectory(42)
        # UMAP
        pt_dim = experiment.pt_dim
        embedding_sizes = range(2, pt_dim + 2) # UMAP doesn't support inverse transform on dim=1
        losses = []
        models = []
        for embedding_size in tqdm(embedding_sizes, position=1):
            model = umap.UMAP(n_neighbors=250, n_components=embedding_size, min_dist=0.0)
            model.fit(traj)
            traj_hat = model.inverse_transform(model.transform(traj))
            loss = mean_squared_error(traj, traj_hat) ** 0.5 # r.m.s
            models.append(model)
            losses.append(loss)
        plt.scatter(embedding_sizes, losses, c=[1 if x < experiment.n_eff else 0 for x in embedding_sizes])
        plt.xlabel("embedding size")
        plt.ylabel("r.m.s. loss")
        plt.xticks(embedding_sizes)
        pltname = f"{experiment.experiment_name} UMAP losses"
        plt.title(pltname)
        plt.savefig(pltname)
        plt.close()
        wandb.log({pltname : wandb.Image(f"{pltname}.png")})
        last_good_embedding_idx = experiment.n_eff - 2
        if last_good_embedding_idx >= 0 and experiment.plot_config is not None:
            model = models[last_good_embedding_idx]
            my_traj_hat = model.inverse_transform(model.transform(traj))
            all_trajs = np.concatenate((traj, my_traj_hat))
            color = np.concatenate((np.zeros(shape=(traj.shape[0], 1)), np.ones(shape=(my_traj_hat.shape[0], 1))))
            exported = np.append(all_trajs, color, axis=1)
            table = wandb.Table(columns=experiment.column_names + ["transformed"], data=exported)
            wandb.log({f"{experiment.experiment_name} UMAP Before/After": table})

In [12]:
umap_experiments()

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
def isomap_experiments():
    for experiment in tqdm(common_experiments, position=0):
        traj = experiment.single_trajectory(42)
        # UMAP
        pt_dim = experiment.pt_dim
        embedding_sizes = range(1, pt_dim + 2)
        losses = []
        models = []
        for embedding_size in tqdm(embedding_sizes, position=1):
            model = Isomap(n_neighbors=50, n_components=embedding_size)
            model.fit(traj)
            loss = model.reconstruction_error()
            models.append(model)
            losses.append(loss)
        plt.scatter(embedding_sizes, losses, c=[1 if x < experiment.n_eff else 0 for x in embedding_sizes])
        plt.xlabel("embedding size")
        plt.ylabel("reconstruction error")
        plt.xticks(embedding_sizes)
        pltname = f"{experiment.experiment_name} Isomap reconstruction errors"
        plt.title(pltname)
        plt.savefig(pltname)
        plt.close()
        wandb.log({pltname : wandb.Image(f"{pltname}.png")})

In [11]:
isomap_experiments()

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
wandb.finish()